In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv('/kaggle/input/otto-group-product-classification-challenge/train.csv')
train

In [ ]:
test = pd.read_csv('/kaggle/input/otto-group-product-classification-challenge/test.csv')
test

In [ ]:
all_data = pd.concat([train,test])
all_data

In [ ]:
all_data_2 = all_data.drop(columns = ['id','target'])
all_data_2

##### Neuralnet : 선형모델을 여러개 쌓는 것(딥러닝)
##### --> 비정형 데이터(텍스트,이미지,오디오....) 분류에 전문으로 쓰임

In [ ]:
'''
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
all_data_3 = ss.fit_transform(all_data_2)
all_data_3
'''

In [ ]:
# 모델점수 올리기 : noise를 잡아주는 RobustScaler 사용
from sklearn.preprocessing import RobustScaler
rs = RobustScaler()
all_data_3 = rs.fit_transform(all_data_2)
all_data_3

In [ ]:
train_2 = all_data_3[:len(train)]
test_2 = all_data_3[len(train):]

In [ ]:
y = pd.get_dummies(train['target'])
y

In [ ]:
# 1) 평가 데이터셋 
# 2) 신경망 기본 모델

from sklearn.model_selection import train_test_split
X_train,X_valid,y_train,y_valid = train_test_split(train_2,y,test_size=0.2,
                                                  random_state=42, stratify=y)
# 모델링 : 네트워크 층을 여러개 설정해줘야 한다!
from keras import *
from keras.layers import *
from keras.callbacks import *
es = EarlyStopping(patience=4,restore_best_weights=True)
# EarlyStopping : 학습중지점
# patience : 몇번정도 점수가 최적점수보다 안좋은걸 참아줄까?
# restore_best_weights : 최적지점에서의 가중치를 모델에 사용하게 하는 옵션

# 층의 목록 가져오기
model = Sequential()
# 입력층 / 은닉층 / 출력층 가져오기
model.add(Dense(1024,activation='relu',input_dim = 93))
# Dense layer : 선형모델
# 1024 : 노드의 수, 몇개의 특징을 추출할 것인가?(컬럼전부를 학습X -> 더늘려 학습 -> 묶어서 같이 본다든지...)
# activation -> 활성화함수(주로 Relu함수 사용)
# input_dim -> 입력층에 들어갈 속성(컬럼 개수만큼 적었음)
model.add(Dense(256,activation='relu'))
model.add(Dense(9,activation='softmax'))
# 9 -> 출력층의 노드개수(y의 컬럼개수)
# 출력층 활성화함수 -> input데이터가 어떤 것이든 0/1 이 출력값으로 나와야하므로 softmax함수 사용
model.compile(metrics=['acc'],loss='categorical_crossentropy',optimizer='adam')
# 모델 선언
# metrics=['acc'] : 정확도를 실시간파악
# loss : 모델이 스스로 학습을 잘하는지 못하는지 체크한것(손실함수->얼마나 에러났는지?)
#      0으로 갈수록 좋음 -> 향상에 도움된 데이터는 유지 / 도움안된 데이터는 폐기
# --> one-hot-encoding시 -> categorical_crossentropy
# --> label encoding시 -> 앞에 sparse붙여서 사용
# optimizer(최적화) = adam -> ???
model.fit(X_train,y_train,validation_data=(X_valid,y_valid),epochs=100,callbacks=[es])
# validation_data = eval_set
# epochs = 학습횟수(callbacks옵션 들어간 후로는 하고싶은만큼 숫자설정해도 좋음!)
result = model.predict_proba(test_2)
# 결과 --> val_loss : 우리 모델의 리더보드 점수

In [ ]:
# 3) 교차검증(test_size=0.2 보다 더 좋은 점수를 얻고싶다!)
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=10,shuffle=True,random_state=42)
# n_splits = 몇등분?
# shuffle = 데이터섞기?
# random_state = 평가셋 고정 --> 모델 점수향상/하락 비교가능!
result = 0

for train_index,valid_index in skf.split(train_2,train['target']):
    X_train = train_2[train_index]  # train_2 독립변수 할당(80%의 인덱스만 가져옴)
    X_valid = train_2[valid_index]  # train_2 독립변수 나머지 20%할당
    y_train = y.iloc[train_index]   # train['target'] 할당(series 형식이므로 iloc표현)
    y_valid = y.iloc[valid_index]   # train['target'] 나머지 20%할당
    
    es = EarlyStopping(patience=5,restore_best_weights=True)
    rl = callbacks.ReduceLROnPlateau(patience=4)
    model = Sequential()
    model.add(Dense(1024,activation='relu',input_dim=93))
    model.add(Dense(256,activation='relu'))
    model.add(Dense(9,activation='softmax'))
    model.compile(metrics=['acc'],optimizer='adam',loss='categorical_crossentropy')
    model.fit(X_train,y_train,validation_data=(X_valid,y_valid),epochs=100,callbacks=[es,rl])
    result += model.predict(test_2) / 10  # 교차검증 n번 한 것을 n으로 나눠줌

result  # 9개의 속성에 대해 확률값으로 있음

In [ ]:
sub = pd.read_csv('/kaggle/input/otto-group-product-classification-challenge/sampleSubmission.csv')
sub

In [ ]:
sub.iloc[:,1:] = result
sub

In [ ]:
sub.to_csv('result.csv',index=False)